## Load the Motion Data

Data is in x and y coordinators for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [ ]:
import numpy as np
import os
from helper_functions import *

data_dir = 'sonic_pi_face/data/'
filename = 'data-18-41-23-5-2017.txt'
data = load_data(data_dir,filename)

In [ ]:
data[4] # 5th frame